# Introductory python tutorial for the synIR library
Welcome to the synIR (/ˈsɪnɚ/) library!

This jupyter notebook showcases the basic python usage of our quantum compilation library.

You can find a detailed detailed explanation of the rust-part of the library in [Rust library overview notebook](Rust%20library%20overview.ipynb).

## Installing synpy library
If you install from the top github folder
```
pip install requirements.txt
pip install synpy/ 
```

If you install from the pypi
```
pip install synpy
```

## Using Qiskit Highlevel synthesis pass

In [1]:
from qiskit.quantum_info import Clifford
from qiskit import QuantumCircuit

Create a simple Qiskit Clifford circuit. Unfortuntately, a Qiskit QuantumCircuit does not know that it is Clifford so we need to make construct it first (`cliff`) and then add it to the a circuit (`qc`).

In [2]:
tmp = QuantumCircuit(2)
tmp.h(0)
tmp.cx(0, 1)
cliff = Clifford(tmp)
qc = QuantumCircuit(2)
qc.append(cliff, (0,1))

Synthesize the circuit directly using the plugin. Here, the plugin takes the Qiskit Clifford object.

In [3]:
from synpy.qiskit.plugin import SynPyCliffordPlugin

In [4]:
direct_circ = SynPyCliffordPlugin().run(cliff, None, None, [])
print(direct_circ)

     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘


Using the Qiskit transpiler by giving the plugin to the transpiler.

In [5]:
from qiskit.transpiler.passes.synthesis.high_level_synthesis import HLSConfig
from qiskit import transpile

In [6]:
hls_config = HLSConfig(clifford=[SynPyCliffordPlugin()])
transpiled_circuit = transpile(qc, hls_config=hls_config)
print(transpiled_circuit)

     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘


Our plugin is also registered as a string in the known transpiler plugins as `"synpy"` so we can use that instead. 

In [7]:
from qiskit.transpiler.passes.synthesis import (
    high_level_synthesis_plugin_names,
)
 
high_level_synthesis_plugin_names("clifford")

['synpy', 'ag', 'bm', 'default', 'greedy', 'layers', 'lnn']

In [8]:
hls_config = HLSConfig(clifford=["synpy"])
transpiled_circuit = transpile(qc, hls_config=hls_config)
print(transpiled_circuit)

     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘


## Using Pauliopt library with synpy

In [9]:
from pauliopt.pauli.pauli_polynomial import PauliPolynomial
from pauliopt.pauli.pauli_gadget import PPhase
from pauliopt.pauli_strings import X, Y, Z, I
from pauliopt.utils import Angle, pi

Make a Toffoli gate using PauliOpt

In [10]:
pp = PauliPolynomial(3)
angle = pi/4
pp >>= PPhase(angle) @ [I, I, X]
pp >>= PPhase(angle) @ [Z, I, I]
pp >>= PPhase(angle) @ [I, Z, I]
pp >>= PPhase(-angle) @ [I, Z, X]
pp >>= PPhase(-angle) @ [Z, Z, I]
pp >>= PPhase(-angle) @ [Z, I, X]
pp >>= PPhase(angle) @ [Z, Z, X]

Synthesize a new circuit using synpy

In [11]:
from synpy.pauliopt.synthesizer import PauliOptSynthesizer

In [12]:
pauliopt_circuit = PauliOptSynthesizer().synthesize(pp)

In [13]:
print(pauliopt_circuit.to_qiskit())

     ┌─────────┐                                                              »
q_0: ┤ Rz(π/4) ├─────────────────────■────────────────────────────■───────────»
     ├─────────┤                   ┌─┴─┐    ┌──────────┐        ┌─┴─┐         »
q_1: ┤ Rz(π/4) ├─────────────■─────┤ X ├────┤ Rz(7π/4) ├──■─────┤ X ├──────■──»
     └──┬───┬──┘┌─────────┐┌─┴─┐┌──┴───┴───┐└──────────┘┌─┴─┐┌──┴───┴───┐┌─┴─┐»
q_2: ───┤ H ├───┤ Rz(π/4) ├┤ X ├┤ Rz(7π/4) ├────────────┤ X ├┤ Rz(7π/4) ├┤ X ├»
        └───┘   └─────────┘└───┘└──────────┘            └───┘└──────────┘└───┘»
«                               
«q_0: ─────────────■────────────
«                  │            
«q_1: ─────────────┼────■───────
«     ┌─────────┐┌─┴─┐┌─┴─┐┌───┐
«q_2: ┤ Rz(π/4) ├┤ X ├┤ X ├┤ H ├
«     └─────────┘└───┘└───┘└───┘


# Future plans
This library is still under construction. If you are looking for specific features that are not yet implemented, please make an issue on the Github page.

Our next plans include, but are not limited to:
- Implement the PSGS algorithm for Pauli Polynomial synthesis (open PR)
- Improve Qiskit transpiler integration, e.g. add PauliExponentialSynthesis
- Circuit-to-PauliExponential parser for generic circuits
- Improve code structure, package structure, and code style.